In [51]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load the food data
food_data = pd.read_csv('./data/Food and Calories.csv')

# Load the user data
user_data = pd.read_csv('./data/User data.csv')

user_data['User_ID'] = range(1, len(user_data) + 1)
food_data['Food_ID'] = range(1, len(food_data) + 1)

# Create a rating column (calories needed) in the user data
def calculate_calories(row):
    if row['Gender'] == 'M':
        bmr = 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] + 5
    else:
        bmr = 10 * row['Weight'] + 6.25 * row['Height'] - 5 * row['Age'] - 161
    activity_level_mapping = {
        'sedentary': 1.2,
        'lightly active': 1.375,
        'moderately active': 1.55,
        'very active': 1.725,
        'extra active': 1.9
    }
    calories_needed = bmr * activity_level_mapping[row['Activity Level']]
    return calories_needed


user_data['Calories'] = user_data.apply(calculate_calories, axis=1)

# Merge user data and food data
merged_data = pd.merge(user_data, food_data, how='cross')

# # Split the data into train and test sets
# train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)

# # Merge the 'Calories' column from user_data into train_data
# train_data = pd.merge(train_data, user_data[['User_ID', 'Calories']], on='User_ID', how='left')

# # Create user and item input tensors
# user_ids = train_data['User_ID'].values
# food_ids = train_data['Food_ID'].values

# # Normalize the ratings (calories)
# min_rating = train_data['Calories'].min()
# max_rating = train_data['Calories'].max()
# ratings = (train_data['Calories'].values - min_rating) / (max_rating - min_rating)

# # Define the neural network model
# embedding_dim = 8  # Number of latent dimensions for user and item embeddings
# num_users = len(user_data)
# num_food = len(food_data)

# # Determine the maximum user and food IDs
# max_user_id = merged_data['User_ID'].max()
# max_food_id = merged_data['Food_ID'].max()

# # Create user and item input tensors
# user_input = tf.keras.layers.Input(shape=(1,))
# food_input = tf.keras.layers.Input(shape=(1,))

# user_embedding = tf.keras.layers.Embedding(max_user_id + 1, embedding_dim, input_length=1)(user_input)
# food_embedding = tf.keras.layers.Embedding(max_food_id + 1, embedding_dim, input_length=1)(food_input)


# # Adjust the learning rate
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# concatenated = tf.keras.layers.Concatenate()([user_embedding, food_embedding])
# dense1 = tf.keras.layers.Dense(32, activation='relu')(concatenated)
# output = tf.keras.layers.Dense(1, activation='linear')(dense1)

# model = tf.keras.Model(inputs=[user_input, food_input], outputs=output)
# model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])


# # Increase the number of epochs and batch size
# model.fit([user_ids, food_ids], ratings, epochs=200, batch_size=128)


,Gender,Age,Weight,Height,Activity Level,User_ID
0,F,56,63.81,183.29,very active,1
1,M,46,48.00,176.62,extra active,2
2,F,32,44.28,196.96,extra active,3
3,F,60,116.68,163.80,sedentary,4
4,F,25,107.77,197.25,extra active,5
...,...,...,...,...,...,...
495,M,37,98.87,174.63,extra active,496
496,F,41,107.85,160.91,lightly active,497
497,F,29,49.82,149.56,moderately active,498
498,F,52,110.12,176.30,moderately active,499


In [ ]:
user_id = 100  # Change this to the desired user ID
num_food_items = food_data.shape[0]

# Get the total calories needed for the user
total_calories_needed = user_data.loc[user_data['User_ID'] == user_id, 'Calories'].values[0]

# Predict ratings (calories) for all food items for the given user
predicted_ratings = model.predict([np.array([user_id] * num_food_items), np.arange(1, num_food_items + 1)])

# Sort the predicted ratings in descending order
sorted_indices = np.argsort(predicted_ratings, axis=0)[::-1]

# Get the top 10 food recommendations based on calories needed
top_food_indices = sorted_indices[:10, 0] + 1  # Add 1 to adjust for 0-based indexing

# Retrieve the food names for the recommended food indices
recommended_food = food_data.loc[top_food_indices.ravel(), 'Food'].tolist()

# Display the recommendations
print("Food Recommendations for User ID:", user_id)
for i, food in enumerate(recommended_food):
    print(f"{i+1}. {food}")